In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import json
import os
import pandas as pd
from datetime import datetime
from pathlib import Path
import numpy as np
from collections import Counter

In [2]:
tweet_dir= Path("../data/processed/park_user_tweets_0530/")
#city = Path("../data/processed/tweets/CO_Denver_0820000.json")

In [3]:
# Load tweets every tweets is labeled with a park name or nan
def build_info_table(tweet_dir, old_index_values = []):
    city_info = []
    for city in tweet_dir.glob("*.json"):
        city_name = city.stem
        #city_summary.index.values 
        if city_name not in old_index_values:
            print("{}".format(city_name))
            with open(city, 'r') as f:
                park_user_tweets = json.load(f)
            n_park_users = len(park_user_tweets)
            total_tweets = 0
            park_tweets = 0
            parks_visisted = []
            for user_tweets in park_user_tweets.values():
                total_tweets += len(user_tweets)
                for tweet in user_tweets:
                    if not pd.isnull(tweet['ParkID']):
                        park_tweets+=1
                        parks_visited.append(tweet['Park_Name'])
            users = list(set([tweet['user']['id_str'] for tweet in tweets]))
            n_users = len(users)
            n_park_tweets = len(park_tweets)
            n_parks_w_tweets = len(set(park_names))
            park_users = list(set([tweet['user']['id_str'] for tweet in park_tweets]))
            n_park_users = len(park_users)
            summary = {"city_file":city,
                       "city": city_name,
                       "tweets":n_tweets,
                       "users":n_users,
                       "park_tweets":n_park_tweets,
                       "park_users":n_park_users,
                       "parks_visited":n_parks_w_tweets}
            city_info.append(summary)
    return city_table

def process_city_table(new_table, old_table):
    new_table = pd.DataFrame.from_records(new_table, index = 'city').sort_values('tweets',ascending=False)
    city_table = pd.concat([old_table, new_table])
    #city_table.to_csv('./results/city_summary.csv')
    return city_table

def build_user_table(tweet_dir):
    city_info = {}
    for city in tweet_dir.glob("*.json"):
        city_name = city.stem
        print("{}".format(city_name))
        with open(city, 'r') as f:
            park_user_tweets = json.load(f)
        n_park_users = len(park_user_tweets)
        total_tweets = 0
        park_tweets = 0
        parks_visited = []
        for user_tweets in park_user_tweets.values():
            total_tweets += len(user_tweets)
            for tweet in user_tweets:
                if not pd.isnull(tweet['ParkID']):
                    park_tweets+=1
                    parks_visited.append(tweet['Park_Name'])
        n_parks_visisted = len(set(parks_visited))
        city_info[city_name] = {"Park Visitors":n_park_users,"Total Tweets":total_tweets,"Park Tweets":park_tweets,"Parks Visited":n_parks_visisted}
    return city_info

In [4]:
tweet_dir= Path("../data/processed/park_user_tweets_0530/")
city_info = build_user_table(tweet_dir)

In [6]:
city_info

{}

In [5]:
order = ['Total Tweets','Park Tweets','% Tweets in Park','Park Visitors', 'Parks Visited']
df = pd.DataFrame.from_dict(city_info, orient='index').sort_values(by='Total Tweets', ascending=False)
df['% Tweets in Park'] = np.round(df['Park Tweets']/df['Total Tweets'],3)*100
df[order].head()

KeyError: 'Total Tweets'

In [66]:
df['city'] = [x[3:-8].replace("_"," ") for x in df.index.values]

In [67]:
pop = pd.read_csv("../data/processed/city_info.csv")
pop.rename({"NAME":"city"}, axis=1, inplace=True)
summary = pd.merge(df,pop[['city','POP2012']],on='city').set_index('city')
summary['Tweets per capita'] = np.round(summary['Total Tweets'] / summary['POP2012'],2)
summary = summary.drop('POP2012',axis=1)
order = ['Total Tweets','Park Tweets','% Tweets in Park','Park Visitors', 'Parks Visited', 'Tweets per capita']
summary['Park Visitors'] = summary['Park Visitors'].apply(lambda x : "{:,}".format(x))
summary['Total Tweets'] = summary['Total Tweets'].apply(lambda x : "{:,}".format(x))
summary['Parks Visited'] = summary['Parks Visited'].apply(lambda x : "{:,}".format(x))
summary['Park Tweets'] = summary['Park Tweets'].apply(lambda x : "{:,}".format(x))
summary = summary[order]


In [68]:
print(summary.to_latex())

\begin{tabular}{lllrllr}
\toprule
{} & Total Tweets & Park Tweets &  \% Tweets in Park & Park Visitors & Parks Visited &  Tweets per capita \\
city          &              &             &                   &               &               &                    \\
\midrule
New York      &    2,892,512 &     213,813 &               7.4 &       113,702 &         1,880 &               0.35 \\
Los Angeles   &    1,215,288 &      53,988 &               4.4 &        36,271 &           540 &               0.32 \\
Philadelphia  &    1,166,125 &      64,857 &               5.6 &        26,287 &           482 &               0.76 \\
Chicago       &    1,130,611 &      66,100 &               5.8 &        36,919 &           872 &               0.41 \\
Houston       &      821,433 &      39,581 &               4.8 &        13,464 &           501 &               0.38 \\
San Antonio   &      589,595 &      23,566 &               4.0 &        12,763 &           268 &               0.43 \\
Washington    &

In [44]:
summary.to_csv('./results/summary_tables/summary_0605.csv')
#df.to_html('table.html')